In [1]:
import json
from glob import glob
from os.path import join as pjoin
import os
import cv2
import numpy as np

from match_elements.GUI_pair import GUIPair
from GUI import GUI
from Operation import Operation

In [2]:
from keras.applications.resnet import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(32, 32, 3))

In [3]:
from paddleocr import PaddleOCR
paddle_ocr = PaddleOCR(use_angle_cls=True, lang='en')

[2022/02/28 10:48:38] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/02/28 10:48:38] root WARNING: lang en is not support in PP-OCRv2, auto switch to version PP-OCR
[2022/02/28 10:48:38] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR
Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/ml/.paddleocr/2.3.0.2/ocr/det/en/en_ppocr_mobile_v2.0_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_s

In [4]:
gui = GUI('data/input/2.jpg')
# gui.element_detection(True, True, True, paddle_cor=paddle_ocr)
gui.load_detection_result()
gui.show_detection_result()

In [5]:
# load operations
testing_data_root = '/home/ml/Data/visual testing/experiments/humanoid-output/adguard-1'

steps_file = pjoin(testing_data_root, 'output.json')
steps = json.load(open(steps_file, 'r'))

operations = []
for step in steps:
    op = Operation(pjoin(testing_data_root, step['tag'] + '.jpg'), step['event_type'], step['bounds'])
    op.shrink_target_element_img()
    op.resize(gui.detection_resize_width, gui.detection_resize_height)        
    op.detect_text(paddle_ocr)
    operations.append(op)
#     op.show_target_ele()

[2022/02/28 10:05:18] root DEBUG: dt_boxes num : 0, elapse : 0.03908991813659668
[2022/02/28 10:05:18] root DEBUG: cls num  : 0, elapse : 0
[2022/02/28 10:05:18] root DEBUG: rec_res num  : 0, elapse : 1.9073486328125e-06
[Text Detection Completed in 0.043 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/28 10:05:18] root DEBUG: dt_boxes num : 3, elapse : 0.014666318893432617
[2022/02/28 10:05:18] root DEBUG: cls num  : 3, elapse : 0.017650127410888672
[2022/02/28 10:05:19] root DEBUG: rec_res num  : 3, elapse : 0.045482635498046875
[Text Detection Completed in 0.083 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/28 10:05:19] root DEBUG: dt_boxes num : 0, elapse : 0.003350973129272461
[2022/02/28 10:05:19] root DEBUG: cls num  : 0, elapse : 0
[2022/02/28 10:05:19] root DEBUG: rec_res num  : 0, elapse : 9.5367431640625e-07
[Text Detection Completed in 0.005 s] Input: data/operation/target.png Output: data/operation/target.js

[Text Detection Completed in 0.014 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/28 10:05:19] root DEBUG: dt_boxes num : 0, elapse : 0.0024061203002929688
[2022/02/28 10:05:19] root DEBUG: cls num  : 0, elapse : 0
[2022/02/28 10:05:19] root DEBUG: rec_res num  : 0, elapse : 7.152557373046875e-07
[Text Detection Completed in 0.004 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/28 10:05:19] root DEBUG: dt_boxes num : 3, elapse : 0.009773731231689453
[2022/02/28 10:05:19] root DEBUG: cls num  : 3, elapse : 0.011065959930419922
[2022/02/28 10:05:19] root DEBUG: rec_res num  : 3, elapse : 0.031097888946533203
[Text Detection Completed in 0.056 s] Input: data/operation/target.png Output: data/operation/target.json
[2022/02/28 10:05:19] root DEBUG: dt_boxes num : 0, elapse : 0.0035562515258789062
[2022/02/28 10:05:19] root DEBUG: cls num  : 0, elapse : 0
[2022/02/28 10:05:19] root DEBUG: rec_res num  : 0, elapse : 4.7683715820

In [6]:
opr = operations[1]
gui.match_elements(target_ele_img=opr.target_element_img, target_ele_text=opr.get_text_content(), resnet_model=resnet_model, show=True)

Match by text


In [21]:
# ********************************************
# *** Streaming Detection with Key Control ***
# ********************************************

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1000)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 500)

while(1):
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)
    
    
    cv2.imshow('camera', frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('s'):
        cv2.imwrite('data/output/temp.jpg', frame)
        gui = GUI('data/output/temp.jpg')
        gui.element_detection(True, True, True, paddle_cor=paddle_ocr)
        cv2.imshow('result', gui.det_result_imgs['merge'])
        print('===================\n')
cap.release()
cv2.destroyAllWindows()

In [4]:
# ***************************
# *** Streaming Detection ***
# ***************************

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1000)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 500)

while(1):
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)
    frame = frame[200:801, :]
    
#     cv2.imshow('camera', frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    cv2.imwrite('data/output/temp.jpg', frame)
    gui = GUI('data/output/temp.jpg')
    gui.element_detection(False, True, True, paddle_cor=paddle_ocr)
    gui.phone_screen_recognition()
    cv2.imshow('camera', gui.det_result_imgs['merge'])
    cv2.imshow('screen', gui.draw_screen(show=False))
    print('===================\n')
cap.release()
cv2.destroyAllWindows()

[Compo Detection Completed in 0.555 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/temp.jpg Output: data/output/merge/temp.jpg

[Compo Detection Completed in 0.577 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/temp.jpg Output: data/output/merge/temp.jpg

[Compo Detection Completed in 0.577 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/temp.jpg Output: data/output/merge/temp.jpg

[Compo Detection Completed in 0.583 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/temp.jpg Output: data/output/merge/temp.jpg

[Compo Detection Completed in 0.577 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/temp.jpg Output: data/output/merge/temp.jpg

[Compo Detection Completed in 0.574 s] Input: data/output/temp.jpg Output: data/output/ip/

[Compo Detection Completed in 0.559 s] Input: data/output/temp.jpg Output: data/output/ip/temp.json
[Merge Completed] Input: data/output/temp.jpg Output: data/output/merge/temp.jpg



In [10]:
# *******************************
# *** Adjust Video Clip Range ***
# *******************************

import cv2

def nothing(x):
    pass

cv2.namedWindow('win')
cv2.createTrackbar('top', 'win', 200, 960, nothing)
cv2.createTrackbar('left', 'win', 0, 540, nothing)
cv2.createTrackbar('bottom', 'win', 800, 960, nothing)
cv2.createTrackbar('right', 'win', 540, 540, nothing)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1000)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 500)

while(1):
    ret, frame = cap.read()
    frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)
    
    top = cv2.getTrackbarPos('top', 'win')
    left = cv2.getTrackbarPos('left', 'win')
    bottom = cv2.getTrackbarPos('bottom', 'win')
    right = cv2.getTrackbarPos('right', 'win')
    
    frame_clip = frame[top:bottom, left:right]
    cv2.imshow('frame', frame)
    cv2.imshow('clip', frame_clip)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [7]:
frame.shape

(960, 540, 3)